<a href="https://colab.research.google.com/github/renatabmagro/ExploratoryAnalysis_DT/blob/main/ExploratoryAnalysis_DecisionTree_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Script para construção dos modelos de Árvore de Decisão - Análise exploratória | Etapa II ***

*Desenvolvido em: Google Colab + Rstudio*

Neste script estão descritos os procedimentos de construção dos modelos de árvore de decisão; Os modelos de árvore de decisão correspondem a Etapa II da análise exploratória dos dados meteorológicos (mensais e sazonais) com relação à produtividade de macieiras.

Foram avaliadas as variáveis meteorológicas para a safra atual (n) e para a safra seguinte (n+1).

> As etapas principais são: 

>> Leitura e organização dos dados. 

>> Construção do modelo de árvore de decisão.
----------------------------------------------

> Foram construídos modelos com os seguintes objetivos: 
  - Avaliação produtividade da safra n (Gala e Fuji)
  - Avaliação produtividade da safra n+1 (Gala e Fuji)

In [ ]:
#Instalação bibliotecas

install.packages("readxl")
library(readxl)
install.packages("tidyr")
install.packages("tidyverse")
library(tidyr)
library(tidyverse)
install.packages("dplyr")
library(dplyr)
install.packages("lubridate")
library(lubridate)

In [ ]:
#Instalação do pacote de Decision tree

  # Pacotes Decision Tree 
install.packages("rpart")
library(rpart)
# Para visualização 
install.packages("rpart.plot")
library(rpart.plot)

In [ ]:
# Leitura dos dados meteorológicos e de produtividade
dados_clima <- read_excel("BD_climaMES_climaESTAC_2011_2020_EN.xlsx")
dados_prod <- read_excel ("BD_producao_cultivar_EN.xlsx")

In [ ]:
# Visualizar dados 
head(dados_clima)
head(dados_prod)

In [ ]:
# Organização dos dados 

#acumulado variáveis meteorológicas em frequencia mensal 
#criar coluna mês/ano
dados_clima$year_month <- floor_date(dados_clima$Data, 
                                  "month")

#head(dados_clima)

In [ ]:
#criar uma nova planilha com os dados agregados 
dados_climaTotal <- dados_clima %>%
  group_by(year_month) %>%
  dplyr::summarize(Tmax = mean(Tmax), Tmin = mean(Tmin), Tmean = mean(Tmean),
                    SRad = mean(SRad), Precip = sum(Precip), GDD = sum(GDD),
                    ET = mean(ET), Chill_Hours = sum(Chill_Hours), 
                    Tmax_summer = mean(Tmax_summer), Tmax_fall = mean(Tmax_fall),  Tmax_winter = mean(Tmax_winter),  Tmax_spring = mean(Tmax_spring),
                    Tmin_summer = mean(Tmin_summer), Tmin_fall = mean(Tmin_fall), Tmin_winter = mean(Tmin_winter), Tmin_spring = mean(Tmin_spring), 
                    Tmed_summer = mean(Tmed_summer), Tmed_fall = mean(Tmed_fall), Tmed_winter = mean (Tmed_winter), Tmed_spring = mean(Tmed_spring), 
                    SRad_summer = mean(SRad_summer),  SRad_fall = mean(SRad_fall),  SRad_winter = mean(SRad_winter),  SRad_spring = mean(SRad_spring),
                    Precip_summer = mean(Precip_summer), Precip_fall = mean(Precip_fall), Precip_winter = mean(Precip_winter), Precip_spring = mean(Precip_spring), 
                    GDD_summer = mean(GDD_summer), GDD_fall = mean(GDD_fall), GDD_winter = mean(GDD_winter), GDD_spring = mean(GDD_spring), 
                    ET_summer = mean(ET_summer),  ET_fall = mean(ET_fall),  ET_winter = mean(ET_winter),  ET_spring = mean(ET_spring),
                    Chill_Hours_summer = mean(Chill_Hours_summer), Chill_Hours_fall = mean(Chill_Hours_fall), Chill_Hours_winter = mean(Chill_Hours_winter), Chill_Hours_spring = mean(Chill_Hours_spring)) %>% as.data.frame()

In [ ]:
head(dados_climaTotal)

In [ ]:
#unir df clima + df produção 

dados_combo <- cbind(dados_climaTotal, dados_prod)
#dados_combo

#selação das variáveis que efetivamente serão utilizadas (~exclusão das variáveis que não serão)
dados_combo <- select(dados_combo, -year_month, -Date, -Chill_Hours_summer)

In [ ]:
glimpse(dados_combo)

In [ ]:
################################################################################

In [ ]:
#######################################################################
## MODELO 1:
### Variável dependente: Next_Yield_GALA
### Variáveis independentes: dados meteorológicos mensais e sazonais
#######################################################################

In [ ]:
# Organização dos dados
## Retirada das variáveis relacionadas à Fuji e à produção da Gala anterior, pois não são necessária nesse momento

dados_comboGala <- select(dados_combo, -NextYield_FUJI, -Yield_FUJI, -Yield_GALA)
head(dados_comboGala)

In [ ]:
# Árvore de decisão 
set.seed(1234)
DT_comboGala = rpart(NextYield_GALA~., data= dados_comboGala, 
                     method="anova", control = rpart.control(cp = 0.03))
#summary(fit.tree)
DT_comboGala

In [ ]:
#visualizar arvore de decisão
rpart.plot(DT_comboGala)

In [ ]:
#variable importance 

split = summary(DT_comboGala)$splits
variable_importanceGala <- DT_comboGala$variable.importance 
barplot(t(DT_comboGala$variable.importance),horiz=TRUE)


In [ ]:
################################################################################

In [ ]:
#######################################################################
## MODELO 2:
### Variável dependente: Yield_GALA
### Variáveis independentes: dados meteorológicos mensais e sazonais
#######################################################################

In [ ]:
# Organização dos dados para o MODELO 2
  # retirada das variáveis relacionadas à produção de Fuji e 'Producao_proximoGALA que não será necessária nesse momento

dados_comboGala_n <- select(dados_combo, -NextYield_FUJI, -Yield_FUJI, -NextYield_GALA)
head(dados_comboGala_n)

In [ ]:
# Árvore de decisão 
set.seed(123)
DT_comboGala_n = rpart(Yield_GALA~., data= dados_comboGala_n, 
                     method="anova", control = rpart.control(cp = 0.035))
#summary(fit.tree)
DT_comboGala_n

#visualizar arvore de decisão
rpart.plot(DT_comboGala_n)

In [ ]:
summary(DT_comboGala_n)

In [ ]:
##########################################################################

In [ ]:
#######################################################################
## MODELO 3:
### Variável dependente: Next_Yield_FUJI
### Variáveis independentes: dados meteorológicos mensais e sazonais
#######################################################################

In [ ]:
# Organização dos dados para construção do MODELO 3
# retirada das variáveis relacionadas à produção de GALA e de Fuji_anterior que não serão necessárias nesse momento

dados_comboFuji <- select(dados_combo, -NextYield_GALA, -Yield_FUJI, -Yield_GALA)
head(dados_comboFuji)

In [ ]:
# Árvore de decisão 
set.seed(123)
DT_comboFuji = rpart(NextYield_FUJI~., data= dados_comboFuji, 
                     method="anova", control = rpart.control(cp = 0.03))
#summary(fit.tree)
DT_comboFuji

In [ ]:
#visualizar arvore de decisão
rpart.plot(DT_comboFuji)

In [ ]:
summary(DT_comboFuji)

In [ ]:
########################################################################

In [ ]:
#######################################################################
## MODELO 4:
### Variável dependente: Yield_FUJI
### Variáveis independentes: dados meteorológicos mensais e sazonais
#######################################################################

In [ ]:
# Organização dos dados para o MODELO 4
# retirada das variáveis relacionadas à Gala e à 'Producao_proximoFuji' que não serão necessárias nesse momento.

dados_comboFuji_n <- select(dados_combo, -NextYield_FUJI, -Yield_GALA, -NextYield_GALA)
head(dados_comboFuji_n)

In [ ]:
# Árvore de decisão 
set.seed(123)
DT_comboFuji_n = rpart(Yield_FUJI~., data= dados_comboFuji_n, 
                     method="anova", control = rpart.control(cp = 0.03))
#summary(fit.tree)
DT_comboFuji_n

#visualizar arvore de decisão
rpart.plot(DT_comboFuji_n)

In [ ]:
summary(DT_comboFuji_n)